<a href="https://colab.research.google.com/github/3000minsu/2023-LG-Aimers/blob/main/4_XGboost(classifier_%2B_onehotencoder).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']
train_y_quality = train_df['Y_Quality']
test = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
over = train_df[train_df['Y_Class'] == 2]['Y_Quality']
normal = train_df[train_df['Y_Class'] == 1]['Y_Quality']
under = train_df[train_df['Y_Class'] == 0]['Y_Quality']


In [ ]:
from sklearn.preprocessing import OneHotEncoder

#LINE one-hot-encoding
ohe_line = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_line = ohe_line.fit_transform(train[['LINE']])
train_line =pd.DataFrame(train_line, columns=['line_' + col for col in ohe_line.categories_[0]])
train = pd.concat((train.drop(columns=['LINE']), train_line), axis=1)

test_line = ohe_line.transform(test[['LINE']])
test_line = pd.DataFrame(test_line, columns=['line_' + col for col in ohe_line.categories_[0]])
test = pd.concat((test.drop(columns=['LINE']), test_line), axis=1)



In [ ]:
ohe_pc = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_pc = ohe_pc.fit_transform(train[['PRODUCT_CODE']])
train_pc =pd.DataFrame(train_pc, columns=['pc_' + col for col in ohe_pc.categories_[0]])
train = pd.concat((train.drop(columns=['PRODUCT_CODE']), train_pc), axis=1)

test_pc = ohe_pc.transform(test[['PRODUCT_CODE']])
test_pc = pd.DataFrame(test_pc, columns=['pc_' + col for col in ohe_pc.categories_[0]])
test = pd.concat((test.drop(columns=['PRODUCT_CODE']), test_pc), axis=1)

In [ ]:
#그래서 하나의 세트에서 모두 결측치인 열은 두 세트에서 모두 삭제 
#data leakage 인가?

invalid=[]
for col in train.columns:
  if(train[col].isnull().all()):
    invalid+=[col]

train = train.drop(columns=invalid)
test = test.drop(columns=invalid)

In [ ]:
RF =  XGBRegressor(objective='reg:squarederror')
RF.fit(train, train_y_quality)

print('Done.')

Done.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=5)
train_y_quality = train_y_quality.values.reshape(-1,1)
kn.fit(train_y_quality, train_y)

KNeighborsClassifier()

In [ ]:
real_preds =[]
preds = RF.predict(test)
for i in range(len(preds)):
  if normal.min() < preds[i] < normal.max():
        preds[i]=1
  elif preds[i] <= under.max():
        preds[i]=0
  elif over.min() <= preds[i]:
       preds[i]=2
  else:
      temp=kn.predict([[preds[i]]])[0]
      preds[i]=temp
     
print('Done.')

Done.


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [ ]:
submit['Y_Class'] = preds

In [ ]:
submit.to_csv('./baseline_submission.csv', index=False)